# Réseaux génératifs

Les réseaux neuronaux récurrents (RNNs) et leurs variantes à cellules à portes, comme les cellules Long Short Term Memory (LSTMs) et les Gated Recurrent Units (GRUs), ont introduit un mécanisme pour la modélisation du langage, c'est-à-dire qu'ils peuvent apprendre l'ordre des mots et fournir des prédictions pour le mot suivant dans une séquence. Cela nous permet d'utiliser les RNNs pour des **tâches génératives**, telles que la génération de texte ordinaire, la traduction automatique et même la génération de légendes pour des images.

Dans l'architecture RNN que nous avons abordée dans l'unité précédente, chaque unité RNN produisait le prochain état caché comme sortie. Cependant, nous pouvons également ajouter une autre sortie à chaque unité récurrente, ce qui nous permettrait de produire une **séquence** (de même longueur que la séquence d'origine). De plus, nous pouvons utiliser des unités RNN qui n'acceptent pas d'entrée à chaque étape, mais qui prennent simplement un vecteur d'état initial, puis produisent une séquence de sorties.

Dans ce notebook, nous allons nous concentrer sur des modèles génératifs simples qui nous aident à générer du texte. Pour simplifier, construisons un **réseau au niveau des caractères**, qui génère du texte lettre par lettre. Pendant l'entraînement, nous devons prendre un corpus de texte et le diviser en séquences de lettres.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Construire un vocabulaire de caractères

Pour créer un réseau génératif au niveau des caractères, nous devons diviser le texte en caractères individuels plutôt qu'en mots. La couche `TextVectorization` que nous avons utilisée auparavant ne peut pas le faire, donc nous avons deux options :

* Charger manuellement le texte et effectuer la tokenisation "à la main", comme dans [cet exemple officiel de Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* Utiliser la classe `Tokenizer` pour la tokenisation au niveau des caractères.

Nous allons opter pour la deuxième option. `Tokenizer` peut également être utilisé pour la tokenisation en mots, ce qui permet de passer facilement de la tokenisation au niveau des caractères à celle au niveau des mots.

Pour effectuer une tokenisation au niveau des caractères, nous devons passer le paramètre `char_level=True` :


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

Nous voulons également utiliser un jeton spécial pour indiquer **fin de séquence**, que nous appellerons `<eos>`. Ajoutons-le manuellement au vocabulaire :


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Entraîner un RNN génératif à créer des titres

La manière dont nous allons entraîner un RNN à générer des titres d'actualités est la suivante. À chaque étape, nous prendrons un titre, qui sera introduit dans un RNN, et pour chaque caractère d'entrée, nous demanderons au réseau de générer le caractère de sortie suivant :

![Image montrant un exemple de génération RNN du mot 'HELLO'.](../../../../../lessons/5-NLP/17-GenerativeNetworks/images/rnn-generate.png)

Pour le dernier caractère de notre séquence, nous demanderons au réseau de générer le token `<eos>`.

La principale différence avec le RNN génératif que nous utilisons ici est que nous prendrons une sortie à chaque étape du RNN, et pas seulement à partir de la cellule finale. Cela peut être réalisé en spécifiant le paramètre `return_sequences` à la cellule RNN.

Ainsi, pendant l'entraînement, une entrée pour le réseau sera une séquence de caractères encodés d'une certaine longueur, et une sortie sera une séquence de la même longueur, mais décalée d'un élément et terminée par `<eos>`. Un minibatch sera constitué de plusieurs de ces séquences, et nous devrons utiliser **padding** pour aligner toutes les séquences.

Créons des fonctions qui transformeront le jeu de données pour nous. Comme nous voulons ajouter du padding aux séquences au niveau du minibatch, nous commencerons par regrouper le jeu de données en appelant `.batch()`, puis nous utiliserons `map` pour effectuer la transformation. Ainsi, la fonction de transformation prendra un minibatch entier comme paramètre :


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Quelques points importants que nous faisons ici :
* Nous commençons par extraire le texte réel du tenseur de chaînes
* `text_to_sequences` convertit la liste de chaînes en une liste de tenseurs d'entiers
* `pad_sequences` remplit ces tenseurs jusqu'à leur longueur maximale
* Enfin, nous encodons tous les caractères en one-hot, tout en effectuant le décalage et en ajoutant `<eos>`. Nous verrons bientôt pourquoi nous avons besoin de caractères encodés en one-hot.

Cependant, cette fonction est **Pythonique**, c'est-à-dire qu'elle ne peut pas être automatiquement traduite en graphe computationnel Tensorflow. Nous obtiendrons des erreurs si nous essayons d'utiliser cette fonction directement dans la fonction `Dataset.map`. Nous devons encapsuler cet appel Pythonique en utilisant le wrapper `py_function` :


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Note** : Différencier entre les fonctions de transformation Pythonic et Tensorflow peut sembler un peu trop complexe, et vous pourriez vous demander pourquoi nous ne transformons pas le dataset en utilisant des fonctions Python standard avant de le passer à `fit`. Bien que cela soit tout à fait possible, utiliser `Dataset.map` présente un énorme avantage, car le pipeline de transformation des données est exécuté via le graphe computationnel de Tensorflow, ce qui permet de tirer parti des calculs sur GPU et de minimiser le besoin de transférer les données entre le CPU et le GPU.

Nous pouvons maintenant construire notre réseau générateur et commencer l'entraînement. Il peut être basé sur n'importe quelle cellule récurrente que nous avons abordée dans l'unité précédente (simple, LSTM ou GRU). Dans notre exemple, nous utiliserons LSTM.

Étant donné que le réseau prend des caractères en entrée et que la taille du vocabulaire est assez petite, nous n'avons pas besoin de couche d'embedding ; une entrée encodée en one-hot peut directement être transmise à la cellule LSTM. La couche de sortie sera un classificateur `Dense` qui convertira la sortie du LSTM en numéros de tokens encodés en one-hot.

De plus, comme nous travaillons avec des séquences de longueur variable, nous pouvons utiliser une couche `Masking` pour créer un masque qui ignorera la partie remplie de la chaîne. Ce n'est pas strictement nécessaire, car nous ne sommes pas particulièrement intéressés par tout ce qui dépasse le token `<eos>`, mais nous l'utiliserons dans le but d'acquérir de l'expérience avec ce type de couche. `input_shape` serait `(None, vocab_size)`, où `None` indique une séquence de longueur variable, et la forme de sortie est également `(None, vocab_size)`, comme vous pouvez le voir dans le `summary` :


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## Génération de sortie

Maintenant que nous avons entraîné le modèle, nous souhaitons l'utiliser pour générer une sortie. Tout d'abord, nous avons besoin d'une méthode pour décoder le texte représenté par une séquence de numéros de tokens. Pour cela, nous pourrions utiliser la fonction `tokenizer.sequences_to_texts` ; cependant, elle ne fonctionne pas bien avec une tokenisation au niveau des caractères. Par conséquent, nous allons prendre un dictionnaire de tokens provenant du tokenizer (appelé `word_index`), construire une correspondance inversée, et écrire notre propre fonction de décodage :


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Commençons par une chaîne `start`, que nous encodons en une séquence `inp`, puis à chaque étape, nous appelons notre réseau pour déduire le caractère suivant.

La sortie du réseau `out` est un vecteur de `vocab_size` éléments représentant les probabilités de chaque jeton, et nous pouvons trouver le numéro du jeton le plus probable en utilisant `argmax`. Nous ajoutons ensuite ce caractère à la liste des jetons générés et poursuivons la génération. Ce processus de génération d'un caractère est répété `size` fois pour produire le nombre requis de caractères, et nous terminons plus tôt si le `eos_token` est rencontré.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## Échantillonnage des résultats pendant l'entraînement

Étant donné que nous ne disposons d'aucune métrique utile comme *l'exactitude*, la seule manière de vérifier que notre modèle s'améliore est de **prélever des exemples** de chaînes générées pendant l'entraînement. Pour ce faire, nous utiliserons des **callbacks**, c'est-à-dire des fonctions que nous pouvons passer à la fonction `fit`, et qui seront appelées périodiquement pendant l'entraînement.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


Cet exemple génère déjà un texte assez bon, mais il peut être amélioré de plusieurs façons :

* **Plus de texte**. Nous avons uniquement utilisé des titres pour notre tâche, mais vous pourriez vouloir expérimenter avec du texte complet. Gardez à l'esprit que les RNN ne sont pas très performants pour gérer de longues séquences, il est donc judicieux soit de les diviser en phrases plus courtes, soit de toujours entraîner sur une longueur de séquence fixe d'une valeur prédéfinie `num_chars` (par exemple, 256). Vous pourriez essayer de modifier l'exemple ci-dessus pour adopter une telle architecture, en vous inspirant du [tutoriel officiel de Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/).

* **LSTM multicouche**. Il est pertinent d'essayer 2 ou 3 couches de cellules LSTM. Comme mentionné dans l'unité précédente, chaque couche de LSTM extrait certains motifs du texte, et dans le cas d'un générateur au niveau des caractères, on peut s'attendre à ce que le niveau inférieur du LSTM soit responsable de l'extraction des syllabes, et les niveaux supérieurs - des mots et des combinaisons de mots. Cela peut être simplement implémenté en passant un paramètre de nombre de couches au constructeur LSTM.

* Vous pourriez également vouloir expérimenter avec **les unités GRU** pour voir lesquelles donnent de meilleurs résultats, ainsi qu'avec **différentes tailles de couches cachées**. Une couche cachée trop grande peut entraîner un surapprentissage (par exemple, le réseau apprendra le texte exact), tandis qu'une taille trop petite pourrait ne pas produire de bons résultats.


## Génération de texte souple et température

Dans la définition précédente de `generate`, nous choisissions toujours le caractère avec la probabilité la plus élevée comme prochain caractère dans le texte généré. Cela avait pour conséquence que le texte "cyclait" souvent entre les mêmes séquences de caractères encore et encore, comme dans cet exemple :  
```
today of the second the company and a second the company ...
```

Cependant, si nous examinons la distribution de probabilité pour le prochain caractère, il se peut que la différence entre quelques probabilités les plus élevées ne soit pas énorme, par exemple, un caractère peut avoir une probabilité de 0,2, un autre de 0,19, etc. Par exemple, en cherchant le prochain caractère dans la séquence '*play*', le caractère suivant pourrait tout aussi bien être un espace ou un **e** (comme dans le mot *player*).

Cela nous amène à la conclusion qu'il n'est pas toujours "juste" de sélectionner le caractère avec la probabilité la plus élevée, car choisir le deuxième plus probable pourrait également conduire à un texte cohérent. Il est plus judicieux de **prélever un échantillon** parmi les caractères en fonction de la distribution de probabilité donnée par la sortie du réseau.

Ce prélèvement peut être effectué à l'aide de la fonction `np.multinomial`, qui implémente ce que l'on appelle la **distribution multinomiale**. Une fonction qui implémente cette génération de texte **souple** est définie ci-dessous :


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

Nous avons introduit un paramètre supplémentaire appelé **température**, qui est utilisé pour indiquer à quel point nous devons nous en tenir à la probabilité la plus élevée. Si la température est de 1,0, nous effectuons un échantillonnage multinomial équitable, et lorsque la température tend vers l'infini - toutes les probabilités deviennent égales, et nous sélectionnons aléatoirement le prochain caractère. Dans l'exemple ci-dessous, nous pouvons observer que le texte devient dénué de sens lorsque nous augmentons trop la température, et il ressemble à un texte "cyclé" généré de manière rigide lorsqu'il se rapproche de 0.



---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction professionnelle effectuée par un humain. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
